In [74]:
import pandas as pd
from pandas_profiling import ProfileReport
import sklearn
import sweetviz as sv

wczytanie zbioru

In [78]:
df = pd.read_csv('./data/titanic.csv')

usuniecie brakujacych rekordow

In [79]:
df = df.dropna()
df = df.reset_index(drop=True)

generowanie raportu sweetviz (pandas profiling cos ma buga)

In [80]:
report = sv.analyze(df)
report.show_html('titanic.html') 

                                             |          | [  0%]   00:00 -> (? left)

/home/jslowik/.pyenv/versions/3.8.12/envs/fusy/lib/python3.8/site-packages/sweetviz/graph_associations.py:223: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_dataframe = new_dataframe.append(pd.Series(empty_row_dict, name=feature))
/home/jslowik/.pyenv/versions/3.8.12/envs/fusy/lib/python3.8/site-packages/sweetviz/graph_associations.py:223: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_dataframe = new_dataframe.append(pd.Series(empty_row_dict, name=feature))
/home/jslowik/.pyenv/versions/3.8.12/envs/fusy/lib/python3.8/site-packages/sweetviz/graph_associations.py:223: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_dataframe = new_dataframe.append(pd.Series(empty_row_dict, name=feature))
/home/jslowik/.pyenv/version

Report titanic.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [82]:
import simpful as sf

# A simple fuzzy model describing how the heating power of a gas burner depends on the oxygen supply.

FS = sf.FuzzySystem()

# Define a linguistic variable.
S_1 = sf.FuzzySet( points=[[0, 1.],  [1., 1.],  [1.5, 0]],          term="low_flow" )
S_2 = sf.FuzzySet( points=[[0.5, 0], [1.5, 1.], [2.5, 1], [3., 0]], term="medium_flow" )
S_3 = sf.FuzzySet( points=[[2., 0],  [2.5, 1.], [3., 1.]],          term="high_flow" )
FS.add_linguistic_variable("OXI", sf.LinguisticVariable( [S_1, S_2, S_3] ))

# Define consequents.
FS.set_crisp_output_value("LOW_POWER", 0)
FS.set_crisp_output_value("MEDIUM_POWER", 25)
FS.set_output_function("HIGH_FUN", "OXI**2")

# Define fuzzy rules.
RULE1 = "IF (OXI IS low_flow) THEN (POWER IS LOW_POWER)"
RULE2 = "IF (OXI IS medium_flow) THEN (POWER IS MEDIUM_POWER)"
RULE3 = "IF (NOT (OXI IS low_flow)) THEN (POWER IS HIGH_FUN)"
FS.add_rules([RULE1, RULE2, RULE3])

# Set antecedents values, perform Sugeno inference and print output values.
FS.set_variable("OXI", .1)
print (FS.Sugeno_inference(['POWER']))

  ____  __  _  _  ____  ____  _  _  __   
 / ___)(  )( \/ )(  _ \(  __)/ )( \(  ) v2.6.2 
 \___ \ )( / \/ \ ) __/ ) _) ) \/ (/ (_/\ 
 (____/(__)\_)(_/(__)  (__)  \____/\____/

 Created by Marco S. Nobile (m.s.nobile@tue.nl)
 and Simone Spolaor (simone.spolaor@unimib.it)

 * Detected Sugeno model type
{'POWER': 0.0}


In [91]:
df[df['Survived']==1][['Age', 'Sex', 'Pclass']].head()

,Age,Sex,Pclass
0,38.0,female,1
1,35.0,female,1
3,4.0,female,3
4,58.0,female,1
5,34.0,male,2


In [92]:
df[df['Survived']==0][['Age', 'Sex', 'Pclass']].head()

,Age,Sex,Pclass
2,54.0,male,1
7,19.0,male,1
9,65.0,male,1
10,45.0,male,1
12,25.0,male,3


In [97]:
from simpful import *

FS = FuzzySystem()


# age
age_TLV = AutoTriangle(3, terms=['young', 'middle', 'old'], universe_of_discourse=[0,80])
FS.add_linguistic_variable("age", age_TLV)


# sex
sex_male = FuzzySet(function=Sigmoid_MF(c=0.5, a=0.1), term="male")
sex_female = FuzzySet(function=InvSigmoid_MF(c=0.5, a=0.1), term="female")
LV1 = LinguisticVariable([sex_male,sex_female], concept="sex", universe_of_discourse=[0,1])
FS.add_linguistic_variable("sex", LV1)

# ticket class
pclass_LV = AutoTriangle(3, terms=['first', 'second', 'third'], universe_of_discourse=[1,3])
FS.add_linguistic_variable("pclass", pclass_LV)

# Define the consequents
FS.set_crisp_output_value("low_probability", 1)
FS.set_crisp_output_value("high_probability", 99)


# Define the fuzzy rules
FS.add_rules([
	"IF (pclass IS first) THEN (death IS low_probability)",
	"IF (pclass IS third) THEN (death IS high_probability)",
	"IF (sex IS female) THEN (death IS low_probability)",
	"IF (sex IS male) THEN (death IS high_probability)",
	"IF (age IS young) THEN (death IS high_probability)",
	"IF (age IS old) THEN (death IS low_probability)",
	])

# pasazer zginal death -probability 77.85 %
FS.set_variable("pclass", 	3) 
FS.set_variable("sex",  	0) 
FS.set_variable("age", 		25)

# pasazer przezyl - 22.37 %
FS.set_variable("pclass", 	1) 
FS.set_variable("sex",  	1) 
FS.set_variable("age", 		54)


death_probability = FS.inference()
death_probability

  ____  __  _  _  ____  ____  _  _  __   
 / ___)(  )( \/ )(  _ \(  __)/ )( \(  ) v2.6.2 
 \___ \ )( / \/ \ ) __/ ) _) ) \/ (/ (_/\ 
 (____/(__)\_)(_/(__)  (__)  \____/\____/

 Created by Marco S. Nobile (m.s.nobile@tue.nl)
 and Simone Spolaor (simone.spolaor@unimib.it)

 * Detected Sugeno model type


{'death': 22.37223185338409}